# PDE Modelling 

## Objectives 
* Make a model that describes cell growth and signalling at the scale of colonies 

## Model considerations 
* Species
    1. Cell density 
    1. Nutrient density 
    1. Environmental AHL (considered equal to intracellular concentration)
    1. Synthase / GFP
    1. Repressor or degradase
* Reactions 
    1. cell growth and diffusion 
        * Cells diffuse very slowly
        * nutrient-dependent growth (from Liu et al 2011, Science) 
        $$  $$ 
    1. Transcriptional activation
        * Basal protein expression -> scaled by repression? probably
        * Activation by internal AHL 
        * Use Hill function $$H(A(t), n, k) = \frac{A(t)^2}{k^n + A(t)^n}$$
        * Activation term, with basal expression and expression rate x
        $$X(A(t), n, k, b, x) = x\frac{A(t)^2}{k^n + A(t)^n} + b$$
    1. Transcriptional repression
        * Assume activation is AND-like, meaning that repression trumps activation
        * Use 'repression' Hill function $$H_n(R(t), n, k) = \frac{k^n}{k^n + R(t)^n}$$
        * Rather than considering protein concentrations within cells, calculate protein concentrations as produced by the bulk of cells. Expression is therefore proportional to cell density.
    1. Dilution and degradation 
        * Assume that GFP/Synthase proteins are degradation tagged
        * Degradase is not tagged, so does not have a degradation term
    1. Diffusion 
        * Here, you're going to use convoultion of the diffusion kernel
        * Diffusion in/out of cell is considered faster than spatial diffusion at these scales
    1. Parameters
        * We are also assuming, for the moment, that each time point is 6 minutes. Parameters with time dimensions shown below may use different units than the parameter from the cited paper.
        * dx: Length modification of diffusion terms. In the compartmental model, diffusion is calculated via Ficks' first law, where the flux between two adjacent compartments is equal to the flux multiplied by the area of the interface between the components :  
        $\frac{\mathrm{d} C}{\mathrm{d} t} $ 
        in continuous form gives up 
        $\Delta C = D \frac{A}{V} \frac{\Delta C}{\Delta x} = D \frac{2.25 \cdot 5 \cdot \mathrm{scale}^2 \mathrm{mm}^2}{\mathrm{scale} \cdot 2.25^2 \cdot 5 \mathrm{mm}^3} \frac{\Delta C \cdot \mathrm{scale}}{2.25 \mathrm{mm}} = \frac{D \Delta C \mathrm{scale}^2}{2.25^2 \mathrm{mm}^2}$. the dx parameter below is the symbol $A$ in this equation.
        * Dc : Diffusion rate for cells. $7\frac{mm^2}{min}$
        * rc : Division rate of cells. $\frac{1.14}{min}$
        * Kn : Half-point of nutrient availability. 75
        * Dn : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * kn : Consumption rate of nutrient by cells
        * Da : Diffusion rate of nutrient. $28\frac{mm^2}{min}$
        * xa : Synthesis rate of AHL. 
        * xs : Expression rate of protein. 
        * ha : Hill coefficient of AHL-inducible expression.
        * ka : Half-point of AHL-inducible expression. 
        * pa : Degradation rate of AHL.
        * leak : Leaky expression rate of protein. 
        


In [ ]:
# imports
from __future__ import division, print_function
import numpy as np
import pandas as pd
import os
import sys
import string
import scipy.integrate as itg
import scipy.optimize as opt
import scipy.ndimage as ndi
import matplotlib as mpl
import matplotlib.pyplot as plt 
import matplotlib.animation as anm
from matplotlib import cm
import skimage.measure
import numba
import gc
from multiprocessing import Pool, Process
import emcee as mc
import seaborn as sns

%load_ext line_profiler

from IPython.display import HTML


rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 24, 
      'xtick.labelsize': 18, 
      'ytick.labelsize': 18, 
      'legend.fontsize': 18,
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)


%matplotlib inline

## 2D Discrete Laplacian

In continuous form : 
$$ U_t = \triangle U - \lambda U $$

In discrete form, for point $i$ : 
$$ \Delta U_i = \sum_{1 = w(i,j)}\omega(i,j)(U_i - U_j) - \lambda U_i $$

Use discrete laplacian approximation w/o diagonals for grid spacing, so that we can have zero-flux  boundary conditions. 

$$ L = 
 \begin{pmatrix}
  0 & 1 & 0 \\
  1 & -4 & 1 \\
  0 & 1 & 0 
 \end{pmatrix} $$

I use a convolution function to calculate the diffusion terms. 

# Helper functions used to define the arenas 
### Needs
* read excel or csv files 
* rescaling arrays and contents 
* convert row/col to array index


* disk function, projects circular areas onto an input grid 
* 

In [ ]:
def disk(A, center, radius):
    h, w = A.shape
    ind_mat = np.zeros((h, w, 2))
    cx, cy = center
    for i in range(h):
        ind_mat[i,:,0] = np.power(np.arange(w) - cx, 2)
    
    for i in range(w):
        ind_mat[:,i,1] = np.power(np.arange(h) - cy, 2)
    
    outmat = (ind_mat[:,:,0] + ind_mat[:,:,1]) < radius**2
    return outmat


let_dict = dict(zip(string.ascii_uppercase, np.arange(0,26)))

scale = 2
scale_s = scale/2 # scale/2
n_w = 48 * scale
n_h = 32 * scale

tmax=1000

species = 12 # rc_cells, cr_cells, nutrients, AHL_c, AHL_r, synthase_c, synthase_r
tup = np.array([species, n_h, n_w])
c1_i, c2_i, c3_i, n_i, rhl_i, cin_i, las_i, rhli_i, cini_i, lasi_i, tetr_i, laci_i = np.arange(species)

# Make empty array, and tolerance arrays
atol = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7
A = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7

# set tolerances
for cell_ind in [c1_i, c2_i, c3_i]:
    atol[cell_ind,:,:]   = 1e-4*np.ones((n_h, n_w), dtype=np.float32)

for signal_ind in [n_i, rhl_i, cin_i, las_i]:
    atol[signal_ind,:,:]   = 1e-4*np.ones((n_h, n_w), dtype=np.float32)
    
for protein_ind in [rhli_i, cini_i, lasi_i, tetr_i, laci_i]:
    atol[protein_ind,:,:]    = 1e-3*np.ones((n_h, n_w), dtype=np.float32)

atol.shape = species*n_h*n_w

rtol = 1e-3

# Set initial conditions
# Nutrients. All at 100
A[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)

# units : L = mm, T = minutes, concentration in nM = moles / mm^3
# Da = 6 - 1.2 E-2
#LEGACY
# Params :    dx,                          Dc,    rc,  Kn,   Dn,   kn,  Da,  xa,  xs,  ha,  ka, 
#p0 = np.array([np.power((scale/2.25),2),   1e-4, 6e-3,  75,  8e-3,  2, 8e-2, 1e3, 2e-0, 2.3, 40,    
          # pa,   leak   od0
#             5e-5, 1e-8, 0.5], dtype=np.float32)

# Params :    ,           dx               Dc,    rc,  Kn,   Dn,   kn,  Da,  xa,  xR,  hR,  kR, 
p0 = np.array([np.power((scale/2.25),2),   1e-4, 6e-3, 75,  3e-2,  2, 3e-2, 1e-1, 2e0, 4, 100,
#xs,  hC, kC, pa,   leak   od0
 2e1, 2.3, 40, 5e-3, 1e-8, 0.5], dtype=np.float32)

# Change parameter values above. The function definitions inherit the parameter values defined here.
dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xR, hR, kR, xs, hC, kC, pa, leak, od0 = p0

#@numba.jit('void(float32[:,:,:],float32[:,:,:])', nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def calc_diffusion(A, D):
    # Middle
    D[:,1:-1,1:-1] = A[:,1:-1, 2:] + A[:,1:-1, :-2] + A[:,:-2, 1:-1] + A[:,2:, 1:-1] - 4*A[:,1:-1, 1:-1]
    # Edges
    D[:,0,1:-1] = A[:,0, 2:] + A[:,0, :-2] + A[:,1, 1:-1] - 3*A[:,0, 1:-1]
    D[:,-1,1:-1] = A[:,-1, 2:] + A[:,-1, :-2] + A[:,-2, 1:-1] - 3*A[:,-1, 1:-1]
    D[:,1:-1,0] = A[:,2:,0] + A[:,:-2,0] + A[:,1:-1,1] - 3*A[:,1:-1,0]
    D[:,1:-1,-1] = A[:,2:,-1] + A[:,:-2,-1] + A[:,1:-1,-2] - 3*A[:,1:-1,-1]
    # Corners
    D[:,0,0] = A[:,0,1] + A[:,1,0] - 2*A[:,0,0]
    D[:,-1,0] = A[:,-1,1] + A[:,-2,0] - 2*A[:,-1,0]
    D[:,0,-1] = A[:,0,-2] + A[:,1,-1] - 2*A[:,0,-1]
    D[:,-1,-1] = A[:,-1,-2] + A[:,-2,-1] - 2*A[:,-1,-1]

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hill(a, n, k):
    h_ma = 1 - (1 / (1 + (a/k)**n))
    return h_ma

#@numba.jit('float32[:,:](float32[:,:],float32,float32)',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def hillN(a, n, k):
    return 1 / (1 + (a/k)**n)

#@numba.jit('void(float32[:,:,:],float32[:,:,:],float32[:,:,:],float32[:,:])',nopython=True, cache=True)
@numba.jit(nopython=True, cache=True)
def calc_f(y, d_y, diff_terms, nut_avail, p0):
    dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xR, hR, kR, xs, hC, kC, pa, leak, od = p0
    calc_diffusion(y, diff_terms)
    
    # Growth term
    nut_avail[:] = hill(y[n_i,:,:], 2, Kn)
    
    # Cell growth
    d_y[c1_i,:,:] = (dx)*Dc*diff_terms[c1_i,:,:] + rc * nut_avail * y[c1_i,:,:]
    d_y[c2_i,:,:] = (dx)*Dc*diff_terms[c2_i,:,:] + rc * nut_avail * y[c2_i,:,:]
    d_y[c3_i,:,:] = (dx)*Dc*diff_terms[c3_i,:,:] + rc * nut_avail * y[c3_i,:,:]
    
    # Nutrient consumption
    d_y[n_i,:,:] = (dx)*Dn*diff_terms[n_i,:,:] - kn * nut_avail * y[:3,:,:].sum(axis=0)
    
    # Signal expression
    d_y[rhl_i,:,:] = (dx)*Da*diff_terms[rhl_i,:,:] + xa * y[rhli_i,:,:] * y[c1_i,:,:] - pa * y[rhl_i,:,:]
    d_y[cin_i,:,:] = (dx)*Da*diff_terms[cin_i,:,:] + xa * y[cini_i,:,:] * y[c2_i,:,:] - pa * y[cin_i,:,:]
    d_y[las_i,:,:] = (dx)*Da*diff_terms[las_i,:,:] + xa * y[lasi_i,:,:] * y[c3_i,:,:] - pa * y[las_i,:,:]
    
    # Protein expression
    d_y[laci_i,:,:] = xR * ((y[c1_i,:,:]>0.05) * hill(y[las_i,:,:], hC, kC) + \
                                        (y[c2_i,:,:]>0.05) * hill(y[rhl_i,:,:], hC, kC) + \
                                        (y[c3_i,:,:]>0.05) * hill(y[cin_i,:,:], hC, kC))
    d_y[tetr_i,:,:] = xR * ((y[c1_i,:,:]>0.05) * hill(y[cin_i,:,:], hC, kC) + \
                                        (y[c2_i,:,:]>0.05) * hill(y[las_i,:,:], hC, kC) + \
                                        (y[c3_i,:,:]>0.05) * hill(y[rhl_i,:,:], hC, kC))
    
    # Synthase expression
    d_y[rhli_i,:,:] = xs * \
        (y[c1_i,:,:]>0.05) * (hill(y[cin_i,:,:], hC, kC)*hillN(y[laci_i,:,:], hR, kR)+\
                               hill(y[las_i,:,:], hC, kC)*hillN(y[tetr_i,:,:], hR, kR)) - pa * y[c1_i,:,:]
    d_y[cini_i,:,:] = xs * \
        (y[c2_i,:,:]>0.05) * (hill(y[las_i,:,:], hC, kC)*hillN(y[laci_i,:,:], hR, kR)+\
                               hill(y[rhl_i,:,:], hC, kC)*hillN(y[tetr_i,:,:], hR, kR)) - pa * y[c1_i,:,:]
    d_y[lasi_i,:,:] = xs * \
        (y[c3_i,:,:]>0.05) * (hill(y[rhl_i,:,:], hC, kC)*hillN(y[laci_i,:,:], hR, kR)+\
                               hill(y[cin_i,:,:], hC, kC)*hillN(y[tetr_i,:,:], hR, kR)) - pa * y[c1_i,:,:]
    

# ODE definition
#@numba.jit('float32[:](float32[:],float32[:],float32[:,:,:],float32[:,:,:],float32[:,:])', nopython=True)
@numba.jit(nopython=True)
def f(y, t, d_y, diff_terms, nut_avail, p0):
    
    y.shape = (species, n_h, n_w)
    calc_f(y, d_y, diff_terms, nut_avail, p0)
    
    return d_y.flatten()

@numba.jit
def f_ivp(t, y, d_y, diff_terms, nut_avail, p0):
    
    y.shape = (species, n_h, n_w)
    calc_f(y, d_y, diff_terms, nut_avail, p0)
    
    return d_y.flatten()


In [ ]:
def prep_initial_condition(cell_spots, ahl_spots, p0, A=A.copy()):
    #dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od = p0
    
    c1_spots, c2_spots, c3_spots = cell_spots
    rhl_spots, cin_spots, las_spots = ahl_spots
    
    for cell_ind, spots in zip([c1_i, c2_i, c3_i],cell_spots):
        cell_arr = np.zeros((n_h, n_w), dtype=np.float32)
        for center in spots:
            cell_arr += disk(cell_arr, scale*np.array(center), scale_s)*0.5
        A[cell_ind, :,:] += ndi.filters.gaussian_filter(cell_arr, scale_s)

    for ahl_ind, spots in zip([rhl_i, cin_i, las_i], ahl_spots):
        ahl_arr = np.zeros((n_h, n_w), dtype=np.float32)
        for center in spots:
            ahl_arr += disk(ahl_arr, scale*np.array(center), scale_s)*0.5
        A[ahl_ind, :,:] += ndi.filters.gaussian_filter(ahl_arr, scale_s)*2.5e3

    return A

    
def sim_omnitray(p0, tmax=tmax, initial_array=A.copy()): 
    args=(np.zeros(initial_array.shape, dtype=np.float32,order='C'), 
          np.zeros(initial_array.shape, dtype=np.float32,order='C'), 
          np.zeros(initial_array.shape[1:], dtype=np.float32,order='C'), 
          p0)
    initial_array.shape = n_h*n_w*species
    print('Starting')
    #out = itg.odeint(f, A, t, args=args, atol=1e-6)
    f_lambda = lambda t, y : f_ivp(t, y, *args)
    # atol = 
    out = itg.solve_ivp(f_lambda, [0, tmax], initial_array, vectorized=True, method='RK23', 
                        atol=atol, rtol=rtol)#, t_eval=np.arange(tmax))
    print('DONE')
    return out

fn_base = "/home/jmp/data/echo_files/20170829_circuit/combo/20170829_combo_ST{}.csv"
fnames = [fn_base.format(strain) 
          for strain in [3,4]
         ]
ahl_fn = "/home/jmp/data/echo_files/20170829_circuit/AHL/20170829_combo_Cin AHL.csv"

#out = sim_omnitray(4, fnames[0], fnames[1], ahl_fn, p0)

# Take a look at one frame
out.resize((t_points,species,n_h,n_w))

print(out.shape)

plt.close('all')
fig, axs = plt.subplots(1,species, figsize=(19,5))
for i in np.arange(species):
    ax = axs[i]
    img = ax.imshow(out[-1,i,:,:], interpolation='none')
    ax.set_xticks([])
    ax.set_yticks([])
    cbar = fig.colorbar(mappable=img, ax=ax)
plt.show()

t_i = np.arange(im_t)[colony_mean[:,0].max() == colony_mean[:,0]]
masks, ls = skimage.measure.label(
    np.logical_xor(im_arr[t_i, 0, :, :] > 0.12, 
                   im_arr[t_i, 1, :, :] > 0.12), 
    connectivity=1, 
    return_num=True)
labels_vec[t_i] = labels
plt.imshow(masks[0])
print(ls)

In [ ]:
cell_spots = [[(3+6*c+18*x, 2+6*y) for x in range(2) for y in range(6)] for c in range(3)]
ahl_spots = [[(3+6+18*1, 2)],[], []]

fn_input = (cell_spots, ahl_spots, p0)

def wrapper(p):
    initial_array = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7
    initial_array[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)
    cell_spots, ahl_spots, p0 = p
    initial_array = prep_initial_condition(cell_spots, [[],[],[]], p0, A=initial_array)
    burn_in = sim_omnitray(p0, tmax=10*60, initial_array=initial_array.copy())
    overnight_t = burn_in.t
    overnight_y = burn_in.y.T
    overnight_y.shape = (len(overnight_t), species, n_h, n_w)
    second_array = prep_initial_condition([[],[],[]],ahl_spots,p0,overnight_y[-1,:,:,:].copy())
    out = sim_omnitray(p0, initial_array=second_array, tmax=100*60)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    out = np.concatenate((overnight_y, exp_y), axis=0)
    t_vec = np.concatenate((overnight_t, exp_t))
    return exp_y, exp_t
    
#with Pool(3) as p:
#    res = p.map(wrapper, fn_inputs)

a_out = wrapper(fn_input)


In [ ]:
col_thresh = 0.1
def write_movie(out, t_vec):
    plt.close('all')
    
    t_points, s, h, w = out.shape
    
    t_list = list(t_vec[-1::-1]) + [-1]
    
    if True:
        frames = t_points
        skip = 1
        im_arr = out[:,:,:,:]
    else:
        frames = 200
        skip = np.int(np.floor(t_points/frames))
        im_arr = out[::skip,:,:,:]
    #frames = len(t)
    
    t, s, h, w = im_arr.shape
    # First set up the figure, the axis, and the plot element we want to animate
    blank_array = np.zeros([n_h, n_w])
    fig, axs = plt.subplots(3,2, figsize=(10,8))
    im_list = [0,0,0,0,0,0,0]
    
    # Plot cell densities
    ax = axs[0,0]
    indxs = [c1_i, c2_i, c3_i]
    vmax = im_arr[:,indxs,:,:].sum(axis=1).max()
    vmin = im_arr[:,indxs,:,:].sum(axis=1).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('cell densities')
    im_list[0] = im
    
    # Plot nutrient densities
    ax = axs[0,1]
    indxs = [n_i]
    vmax = im_arr[:,indxs[0],:,:].max()
    vmin = im_arr[:,indxs[0],:,:].min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('nutrient')
    im_list[1] = im
    
    # Plot synthase densities
    ax = axs[1,0]
    indxs = [rhli_i, cini_i, lasi_i]
    vmax = im_arr[:,indxs,:,:].sum(axis=1).max()
    vmin = im_arr[:,indxs,:,:].sum(axis=1).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('synthases')
    im_list[2] = im
    
    # Plot AHL densities
    ax = axs[1,1]
    indxs = [rhl_i, cin_i, las_i]
    vmax = im_arr[:,indxs,:,:].sum(axis=1).max()
    vmin = im_arr[:,indxs,:,:].sum(axis=1).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('AHLs')
    im_list[3] = im
    
    # Plot colony masks
    ax = axs[2,0]
    indxs = [c1_i, c2_i, c3_i]
    vmax = 3
    vmin = 0
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('masks')
    im_list[4] = im
    
    # Plot repressors 
    ax = axs[2,1]
    indxs = [tetr_i, laci_i]
    vmax = im_arr[:,indxs,:,:].sum(axis=1).max()
    vmin = im_arr[:,indxs,:,:].sum(axis=1).min()
    im = ax.imshow(blank_array, animated=True, vmax=vmax, vmin=vmin, interpolation='none')
    cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title('synthases in colonies')
    im_list[5] = im
    
    fig.tight_layout()
    
    # animation function.  This is called sequentially
    frame_arr = np.zeros(im_arr.shape[2:])
    def animate(i):
        
        if i < t_list[-1]:
            return 0
        
        t_list.pop()
        
        # Plot cell densities
        ax = axs[0,0]
        indxs = [c1_i, c2_i, c3_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[0].set_array(frame_arr)

        # Plot nutrient densities
        ax = axs[0,1]
        indxs = [n_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[1].set_array(frame_arr)

        # Plot synthase densities
        ax = axs[1,0]
        indxs = [rhli_i, cini_i, lasi_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[2].set_array(frame_arr)

        # Plot synthase densities
        ax = axs[1,1]
        indxs = [rhl_i, cin_i, las_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[3].set_array(frame_arr)

        # Plot colony masks
        ax = axs[2,0]
        indxs = [c1_i, c2_i, c3_i]
        frame_arr  = (im_arr[i, indxs[0], :, :] > col_thresh)*1
        frame_arr += (im_arr[i, indxs[1], :, :] > col_thresh)*2
        frame_arr += (im_arr[i, indxs[2], :, :] > col_thresh)*3
        im_list[4].set_array(frame_arr)
        
        # Plot colony masks
        ax = axs[2,1]
        indxs = [tetr_i, laci_i]
        frame_arr = im_arr[i,indxs,:,:].sum(axis=0)
        im_list[5].set_array(frame_arr)
        
        #return im_list,

    # call the animator.  blit=True means only re-draw the parts that have changed.
    anim = anm.FuncAnimation(fig, animate, interval=50, frames=frames)


    # Set up formatting for the movie files
    #Writer = anm.writers['ffmpeg_file']
    #writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=900, extra_args=['-vcodec', 'libx264'])

    # save the animation as an mp4.  This requires ffmpeg or mencoder to be
    # installed.  The extra_args ensure that the x264 codec is used, so that
    # the video can be embedded in html5.  You may need to adjust this for
    # your system: for more information, see
    # http://matplotlib.sourceforge.net/api/animation_api.html
    #anim.save('animation_{}.mp4'.format(fn), extra_args=['-vcodec', 'libx264'], dpi=50, writer=writer)
    #plt.close('all')


    #anim.save('animation_{}.mp4'.format(fn), writer=writer)
    plt.close('all')
    return anim
    #HTML(anim.to_html5_video())
    

out_names = ["a", "b", "c"]
out, t_vec = a_out

anim = write_movie(out, t_vec)
HTML(anim.to_html5_video())

In [ ]:
jd = pd.read_csv('20170829_pos1_wCtime.csv')
cols = jd.columns
data_cols = cols[3:9]
m1, m2 = jd[data_cols[::2]].dropna(axis=0).T.values.max(), jd[data_cols[1::2]].dropna(axis=0).T.values.max()
plt.figure(figsize=(20,8))
clean_fit_dict = dict()
for col in data_cols[::2]:
    y, t = list(jd[[col, 'Time elapsed']].dropna(axis=0).T.values)
    y = (y-y.min())/(m1-y.min())
    plt.plot(t, y,'.', label=col)
    fit = np.polyfit(t, y, 5)
    fitfn = np.poly1d(fit)
    clean_fit_dict[col] = fitfn
    plt.plot(t, fitfn(t),'k')
    
for col in data_cols[1::2]:
    y, t = list(jd[[col, 'Time elapsed']].dropna(axis=0).T.values)
    y = (y-y.min())/(m2-y.min())
    plt.plot(t, y,'.', label=col)
    fit = np.polyfit(t, y, 5)
    fitfn = np.poly1d(fit)
    clean_fit_dict[col] = fitfn
    plt.plot(t, fitfn(t),'k')    
    
clean_fit_vec = [clean_fit_dict[key] for key in data_cols]

plt.plot([0, 70], [0.18, 0.18], '--k')
plt.legend()

In [ ]:
n_colonies = 6
def get_traces(out):
    t, s, h, w = out.shape
    colony_mean = np.zeros((t, n_colonies))
    frame_arr = np.zeros((n_h,n_w))
    for t_i in np.arange(t):
        frame_arr = np.logical_xor(out[t_i, cr_i, :, :] > col_thresh, out[t_i, rc_i, :, :] > col_thresh)
        frame_arr[:,:np.int(n_w/2)] = 0
        masks, n_l = skimage.measure.label(
            frame_arr,
            connectivity=1, 
            return_num=True)
        for l in range(0,n_colonies,2):
            colony_mean[t_i, l] = np.mean((masks==(l+1))*(out[t_i, cini_i,:,:]))
        for l in range(1,n_colonies,2):
            colony_mean[t_i, l] = np.mean((masks==(l+1))*(out[t_i, rhli_i,:,:]))
    return colony_mean

def fit_wrapper(p):
    rc_spots, cr_spots, ahl_spots, p0 = p
    second_array = prep_initial_condition([],[],ahl_spots,p0,overnight_y[-1,:,:,:].copy())
    out = sim_omnitray(p0, initial_array=second_array, tmax=70*60)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    out = np.concatenate((overnight_y, exp_y), axis=0)
    t_vec = np.concatenate((overnight_t, exp_t))
    return exp_y, exp_t

def sim_wrapper(p_in):
    dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od0 = p0
    xa, xs, hR, kR, hC, kC, pa, leak, od = p_in
    p_in = dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od
    return fit_wrapper((dest_wells[0][0], dest_wells[0][1], ahl_centers, p_in))

def residuals(p_in):
    if np.any(np.array(p_in) < 0):
        return np.inf
    y, t = sim_wrapper(p_in)
    sim_traces = get_traces(y)
    sim_traces = sim_traces / sim_traces.max()
    res = 0
    for lab in range(6):
        sim_y = sim_traces[:,lab] / sim_traces[:,lab].max()
        data_trace = clean_fit_vec[lab](t/60)
        res += np.sum(np.power(sim_traces[:,lab] - data_trace, 2))
    return res


In [ ]:
if True:
    fig, axs = plt.subplots(2,1, figsize=(8,11))
    rfp_colors = cm.Reds(np.linspace(0.4,0.9,3))
    gfp_colors = cm.Greens(np.linspace(0.4,0.9,3))
    color_dict = dict(zip([5,3,1,4,2,0],np.concatenate((gfp_colors,rfp_colors))))
    out, t_vec = a_out
    keep_inds = t_vec < 71*60
    t_vec = t_vec[keep_inds]
    out = out[keep_inds,:,:,:]
    t_vec = t_vec / 60
    plot_labels = ["Colony {}".format(i) for i in np.arange(6,0,-1) ]
    traces = get_traces(out)
    traces = traces / traces.max()
    print(traces.shape)
    for colony in range(n_colonies-1,-1,-1):
        axs[0].plot(t_vec,traces[:,colony], color=color_dict[colony], label=plot_labels[colony], linewidth=3)
    plt.figure()
    t_crossings = np.zeros(6)
    threshold = (traces[25,-1])
    axs[0].plot(t_vec[[0,-1]], [threshold, threshold],'k--', label='Threshold')
    #t_vec = t.copy()
    #t_vec.shape
    for colony in range(n_colonies):
        above_threshold = traces[:,colony] > threshold
        t_crossings[colony] = t_vec[above_threshold].min()
    col_x = np.arange(n_colonies) # * 2.25*3*np.sqrt(2)
    col_y = t_crossings[::-1]
    lin_fn = lambda x, m, b: m*x + b
    opt_out = opt.curve_fit(lin_fn, col_x, col_y)
    corrcoef = np.corrcoef(col_x, col_y)[0][1]
    axs[1].plot(col_x, col_y,'.', ms=20, label='Data points')
    axs[1].plot(col_x, lin_fn(col_x, *opt_out[0]),'r-', ms=20, label='Fitted line')
    axs[1].text(2.5, 5, "r = {:0.3f}\nSlope = {:0.1f} hr/colony".format(corrcoef,opt_out[0][0]), {'size':20})
    axs[1].set_xticks(np.arange(n_colonies))
    axs[1].set_xticklabels(plot_labels[::-1])#, rotation=-10, ha='left')
    axs[1].set_title('Estimating simulated cascade velocity')
    axs[1].set_ylabel('Time to pass threshold (hours)')
    axs[1].legend()
    axs[0].legend()
    axs[0].set_xlabel('Time (hours)')
    axs[0].set_ylabel('Normalized mean\ncolony fluorescence', wrap=True)
    axs[0].set_title('Simulation results')
    fig.tight_layout()
    fig.savefig('sim_analysis.pdf')
    print("velocity: {}cm/hour".format((1/1e1)*9.55/np.mean(np.diff(t_crossings[-1::-1]))))

In [ ]:
def read_centers(fn):
    mat = pd.read_csv(fn)
    well_strs = list(mat[['    Destination Well']].values[:,0])
    centers = [(int(w[1:]), let_dict[w[:1]]) for w in well_strs]
    return centers
    

fn_base = "/home/jmp/data/echo_files/20170829_circuit/"

ahl_ends =  ["AHL/20170829_circle_Cin AHL.csv",
             "AHL/20170829_combo_Cin AHL.csv",
             "AHL/20170829_abc{}_Cin AHL.csv"]
 
cell_ends = ["circle/20170829_circle{}_ST{}.csv",
              "combo/20170829_combo_ST{}.csv",
              "no AiiA/20170829_{}{}_ST{}.csv",
              "weak AiiA/20170829_{}{}_ST{}.csv"]

circle_spots = [
    [ read_centers(fn_base + cell_ends[0].format(ind, strain)) for strain in [3,4] ] + \
    [read_centers(fn_base + ahl_ends[0]), p0, "circle{}.html".format(ind)]
        for ind in [1,2,3] ]

noaiia_spots = [
    [read_centers(fn_base + cell_ends[2].format(let,space,strain)) for strain in [3,4] ] + \
    [read_centers(fn_base + ahl_ends[2].format(space)), p0, "noaiia_{}{}.html".format(let,space)]
        for let in ['a','b','c'] for space in [1,2,3] ]

all_spots = circle_spots + noaiia_spots



In [ ]:
def write_wrapper(p):
    initial_array = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7
    initial_array[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)
    rc_spots, cr_spots, ahl_spots, p0, write_name = p
    initial_array = prep_initial_condition(rc_spots, cr_spots, [], p0, A=initial_array)
    burn_in = sim_omnitray(p0, tmax=18*60, initial_array=initial_array.copy())
    overnight_t = burn_in.t
    overnight_y = burn_in.y.T
    overnight_y.shape = (len(overnight_t), species, n_h, n_w)
    second_array = prep_initial_condition([],[],ahl_spots,p0,overnight_y[-1,:,:,:].copy())
    out = sim_omnitray(p0, initial_array=second_array, tmax=4*24*60)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    out = np.concatenate((overnight_y, exp_y), axis=0)
    t_vec = np.concatenate((overnight_t, exp_t))
    anim = write_movie(out, t_vec)
    Writer = anm.writers['html']
    writer = Writer(fps=30, metadata=dict(artist='Me'), bitrate=900, extra_args=['-vcodec', 'libx264'])
    
    anim.save(write_name, writer=writer)
    plt.close('all')
    
#with Pool(4) as p:
#    p.map(write_wrapper, all_spots)

#import pymc3 as pmc
import emcee as mc


initial_array = np.zeros((species, n_h, n_w), dtype=np.float32,order='C')# + 1e-7
initial_array[n_i,:,:] = 100*np.ones((n_h, n_w), dtype=np.float32)
rc_spots, cr_spots, ahl_spots, p0, write_name = dest_wells[0][0], dest_wells[0][1], ahl_centers, p0, ''
initial_array = prep_initial_condition(rc_spots, cr_spots, [], p0, A=initial_array)
burn_in = sim_omnitray(p0, tmax=18*60, initial_array=initial_array.copy())
overnight_t = burn_in.t
overnight_y = burn_in.y.T
overnight_y.shape = (len(overnight_t), species, n_h, n_w)
second_array = prep_initial_condition([],[],ahl_spots,p0,overnight_y[-1,:,:,:].copy())
out = sim_omnitray(p0, initial_array=second_array, tmax=4*24*60)
exp_t = out.t
exp_y = out.y.T
exp_y.shape = (len(exp_t), species, n_h, n_w)
out = np.concatenate((overnight_y, exp_y), axis=0)
t_vec = np.concatenate((overnight_t, exp_t))

def fit_wrapper(p):
    rc_spots, cr_spots, ahl_spots, p0 = p
    second_array = prep_initial_condition([],[],ahl_spots,p0,overnight_y[-1,:,:,:].copy())
    out = sim_omnitray(p0, initial_array=second_array, tmax=75*60)
    exp_t = out.t
    exp_y = out.y.T
    exp_y.shape = (len(exp_t), species, n_h, n_w)
    return exp_y, exp_t

def sim_wrapper(p_in):
    dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od0 = p0
    xa, xs, hR, kR, hC, kC, pa, leak, od = p_in
    p_in = dx, Dc,  rc,    Kn,  Dn,   kn, Da, xa, xs, hR, kR, hC, kC, pa, leak, od
    return fit_wrapper((dest_wells[0][0], dest_wells[0][1], ahl_centers, p_in))

def residuals(p_in):
    if np.any(np.array(p_in) < 0):
        return np.inf
    y, t = sim_wrapper(p_in)
    sim_traces = get_traces(y)
    sim_traces = sim_traces / sim_traces.max()
    res = 0
    for lab in range(6):
        sim_y = sim_traces[:,lab]
        data_trace = clean_fit_vec[lab](t/60)
        res += np.sum(np.power(sim_traces[:,lab] - data_trace, 2))
    return res


p_in = xa, xs, hR, kR, hC, kC, pa, leak, od0
residuals(p_in)

p_in = xa, xs, hR, kR, hC, kC, pa, leak, od0

def lnprob(p_in):
    resis = residuals(p_in)
    return -resis
    
ndim, nwalkers = len(p_in), 100

pos = [p_in + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
sampler = mc.EnsembleSampler(nwalkers, ndim, lnprob)
trace = sampler.run_mcmc(pos, 500)

In [ ]:
#flatchain = sampler.flatchain[sampler.flatchain[:,0]>0,:]
#fc = pd.DataFrame(flatchain)
#fc.to_csv('flatchain.csv',index=False)

In [ ]:
#fc = pd.read_csv('flatchain.csv', index_col=None)
#fc.columns = np.array(["xa" , "xs" , "hR" , "kR" , "hC" , "kC" , "pa" , "leak" , "od"])
#g = sns.pairplot(fc)
#g.map_diag(sns.kdeplot)
#g.map_offdiag(sns.kdeplot, cmap="Blues_d", n_levels=6);

In [ ]:
out, t_vec = a_out
t_points, s, h, w = out.shape
col_thresh = 0.1

t_list = list(t_vec[-1::-1]) + [-1]

if True:
    frames = t_points
    skip = 1
    im_arr = out[:,:,:,:]
else:
    frames = 500
    skip = np.int(np.floor(t_points/frames))
    im_arr = out[::skip,:,:,:]
#frames = len(t)

if False:
    fig_name = 'rot_firstframe.pdf'
    t_i=0
else:
    fig_name = 'rot_lastframe.pdf'
    t_i=-1
    

t, s, h, w = im_arr.shape
# First set up the figure, the axis, and the plot element we want to animate
blank_array = np.zeros([n_h, n_w])
fig, axs = plt.subplots(3,2, figsize=(10,8))
im_list = [0,0,0,0,0,0,0]

# Plot cell densities
ax = axs[0,0]
indxs = [c1_i, c2_i, c3_i]
vmax = im_arr[:,indxs[0],:,:].max()
vmin = im_arr[:,indxs[0],:,:].min()
im = ax.imshow(im_arr[t_i,indxs,:,:].sum(axis=0), vmax=vmax, vmin=vmin, interpolation='none')
cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Cell concentration')
im_list[0] = im

# Plot nutrient densities
ax = axs[0,1]
indxs = [n_i]
vmax = im_arr[:,indxs[0],:,:].max()
vmin = im_arr[:,indxs[0],:,:].min()
im = ax.imshow(im_arr[t_i,indxs,:,:].sum(axis=0), vmax=vmax, vmin=vmin, interpolation='none')
cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Nutrient concentration')
im_list[1] = im

# Plot synthase densities
ax = axs[1,0]
indxs = [rhli_i, cini_i, lasi_i]
vmax = im_arr[:,indxs,:,:].sum(axis=1).max()
vmin = im_arr[:,indxs,:,:].sum(axis=1).min()
im = ax.imshow(im_arr[t_i,indxs,:,:].sum(axis=0), vmax=vmax, vmin=vmin, interpolation='none')
cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Synthase quantity')
im_list[2] = im

# Plot AHL densities
ax = axs[1,1]
indxs = [rhl_i, cin_i, las_i]
vmax = (im_arr[:,indxs[0],:,:]+im_arr[:,indxs[1],:,:]).max()
vmin = (im_arr[:,indxs[0],:,:]+im_arr[:,indxs[1],:,:]).min()
im = ax.imshow(im_arr[t_i,indxs,:,:].sum(axis=0), vmax=vmax, vmin=vmin, interpolation='none')
cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('AHL (nM)')
im_list[3] = im

# Plot colony masks
ax = axs[2,0]
indxs = [c1_i, c2_i, c3_i]
vmax = 6
vmin = 0
frame_arr = im_arr[t_i,indxs,:,:].sum(axis=0) > col_thresh
#frame_arr[:,:np.int(n_w/2)] = 0
mask, labls = skimage.measure.label(frame_arr,return_num=True, connectivity=1) 
mask = (7 - mask)*frame_arr
cmap = cbar.cmap
color_list = cmap(np.linspace(0,1,8))
color_list[0] = (0.5,0.5,0.5,1)
cmap = mpl.colors.ListedColormap(color_list)
bounds=np.linspace(0,7,8)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
im = ax.imshow(mask, vmax=vmax, vmin=vmin, interpolation='none', cmap=cmap, norm=norm)
cbar = fig.colorbar(im, ax=ax, ticks=[1, 6])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Colony identities')
im_list[4] = im

# Plot synthases within masks
ax = axs[2,1]
indxs = [rhli_i, cini_i, las_i]
vmax = (im_arr[:,indxs[0],:,:]+im_arr[:,indxs[1],:,:]).max()
vmin = (im_arr[:,indxs[0],:,:]+im_arr[:,indxs[1],:,:]).min()
im = ax.imshow(frame_arr*im_arr[t_i,indxs,:,:].sum(axis=0), vmax=vmax, vmin=vmin, interpolation='none')
cbar = fig.colorbar(im, ax=ax, ticks=[vmin, vmax])
ax.set_xticks([])
ax.set_yticks([])
ax.set_title('Synthases in colonies')
im_list[5] = im

fig.tight_layout()
fig.savefig(fig_name)

In [ ]:
frame_arr.shape